In [267]:
import os
import json
import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities
from keras.models import Sequential
from keras.layers import LSTM


In [268]:
file=open("../input/conversation.json")
data =json.load(file)
c=data["conversations"]
print (c)
model=Word2Vec(c, size=100, window=5, min_count=1, workers=4)
model=gensim.models.Word2Vec.load('word2vec.bin')
print (model)

[['Good morning, how are you?', 'I am doing well, how about you?', "I'm also good.", "That's good to hear.", 'Yes it is.'], ['Hello', 'Hi', 'How are you doing?', 'I am doing well.', 'That is good to hear', 'Yes it is.', 'Can I help you with anything?', 'Yes, I have a question.', 'What is your question?', 'Could I borrow a cup of sugar?', "I'm sorry, but I don't have any.", 'Thank you anyway', 'No problem'], ['How are you doing?', 'I am doing well, how about you?', 'I am also good.', "That's good."], ['Have you heard the news?', 'What good news?'], ['What is your favorite book?', "I can't read.", "So what's your favorite color?", 'Blue'], ['Who are you?', 'Who? Who is but a form following the function of what', 'What are you then?', 'A man in a mask.', 'I can see that.', "It's not your powers of observation I doubt, but merely the paradoxical nature of asking a masked man who is. But tell me, do you like music?", 'I like seeing movies.', 'What kind of movies do you like?', 'Alice in Won

In [269]:
#break data into question and answer pair
q=[]
a=[]
for i in range(len(c)):
    for j in range(len(c[i])):
        if j<len(c[i])-1:
            q.append(c[i][j]) #questions
            a.append(c[i][j+1]) #its answers
#tokenize....seperate 
tk_q=[]
tk_a=[]
for i in range(len(q)):
    tk_q.append(nltk.word_tokenize(q[i].lower()))
    tk_a.append(nltk.word_tokenize(a[i].lower()))
tk_q

[['good', 'morning', ',', 'how', 'are', 'you', '?'],
 ['i', 'am', 'doing', 'well', ',', 'how', 'about', 'you', '?'],
 ['i', "'m", 'also', 'good', '.'],
 ['that', "'s", 'good', 'to', 'hear', '.'],
 ['hello'],
 ['hi'],
 ['how', 'are', 'you', 'doing', '?'],
 ['i', 'am', 'doing', 'well', '.'],
 ['that', 'is', 'good', 'to', 'hear'],
 ['yes', 'it', 'is', '.'],
 ['can', 'i', 'help', 'you', 'with', 'anything', '?'],
 ['yes', ',', 'i', 'have', 'a', 'question', '.'],
 ['what', 'is', 'your', 'question', '?'],
 ['could', 'i', 'borrow', 'a', 'cup', 'of', 'sugar', '?'],
 ['i', "'m", 'sorry', ',', 'but', 'i', 'do', "n't", 'have', 'any', '.'],
 ['thank', 'you', 'anyway'],
 ['how', 'are', 'you', 'doing', '?'],
 ['i', 'am', 'doing', 'well', ',', 'how', 'about', 'you', '?'],
 ['i', 'am', 'also', 'good', '.'],
 ['have', 'you', 'heard', 'the', 'news', '?'],
 ['what', 'is', 'your', 'favorite', 'book', '?'],
 ['i', 'ca', "n't", 'read', '.'],
 ['so', 'what', "'s", 'your', 'favorite', 'color', '?'],
 ['who', '

None


In [272]:
arr1=np.ones((300,),dtype=np.float32) 
#vectors of q and a
vec_q=[]
for sent in tk_q:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_q.append(sentvec)
vec_a=[]
for sent in tk_a:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_a.append(sentvec)
    
#making 111s in blank spaces and adding 111s to vec_q and vec_a to make lengths equal
for tok_sent in vec_q:
    tok_sent[14:]=[]
    tok_sent.append(arr1)
for tok_sent in vec_q:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(arr1)

            
for tok_sent in vec_a:
    tok_sent[14:]=[]
    tok_sent.append(arr1)
for tok_sent in vec_a:
    if len(tok_sent)<15:
        for i in range(15-len(tok_sent)):
            tok_sent.append(arr1)


In [283]:
#np arrays
q=np.array(vec_q,dtype=np.float64)
a=np.array(vec_a,dtype=np.float64)   
print(q.shape)
print(a.shape)
x_train,x_test, y_train,y_test = train_test_split(q,a,test_size=0.2, random_state=1)
model=Sequential()
model.add(LSTM(output_dim=300,input_shape=x_train.shape[1:],return_sequences=True,activation='sigmoid')) #return seq=True
model.add(LSTM(300,return_sequences=True,activation='sigmoid'))
model.add(LSTM(300,return_sequences=True,activation='sigmoid'))
model.add(LSTM(300,return_sequences=True,activation='sigmoid'))
model.add(LSTM(300,return_sequences=True,activation='sigmoid'))
model.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy'])

model.fit(x_train,y_train,epochs=100,validation_split=0.2)

(86, 15, 300)
(86, 15, 300)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(15, 300), return_sequences=True, activation="sigmoid", units=300)`
  


Train on 54 samples, validate on 14 samples
Epoch 1/100
54/54 [==============================] - 6s 104ms/step - loss: -0.9947 - acc: 0.0000e+00 - val_loss: -0.9985 - val_acc: 0.0000e+00
Epoch 2/100
54/54 [==============================] - 0s 6ms/step - loss: -0.9982 - acc: 0.0000e+00 - val_loss: -0.9975 - val_acc: 0.0000e+00
Epoch 3/100
54/54 [==============================] - 0s 6ms/step - loss: -0.9976 - acc: 0.0000e+00 - val_loss: -0.9985 - val_acc: 0.0000e+00
Epoch 4/100
54/54 [==============================] - 0s 6ms/step - loss: -0.9986 - acc: 0.0000e+00 - val_loss: -0.9991 - val_acc: 0.0000e+00
Epoch 5/100
54/54 [==============================] - 0s 6ms/step - loss: -0.9991 - acc: 0.0000e+00 - val_loss: -0.9992 - val_acc: 0.0000e+00
Epoch 6/100
54/54 [==============================] - 0s 6ms/step - loss: -0.9992 - acc: 0.0000e+00 - val_loss: -0.9993 - val_acc: 0.0000e+00
Epoch 7/100
54/54 [==============================] - 0s 6ms/step - loss: -0.9993 - acc: 0.0000e+00 - val_los

54/54 [==============================] - 0s 6ms/step - loss: -1.0000 - acc: 0.0000e+00 - val_loss: -1.0000 - val_acc: 0.0000e+00
Epoch 59/100
54/54 [==============================] - 0s 6ms/step - loss: -1.0000 - acc: 0.0000e+00 - val_loss: -1.0000 - val_acc: 0.0000e+00
Epoch 60/100
54/54 [==============================] - 0s 6ms/step - loss: -1.0000 - acc: 0.0000e+00 - val_loss: -1.0000 - val_acc: 0.0000e+00
Epoch 61/100
54/54 [==============================] - 0s 6ms/step - loss: -1.0000 - acc: 0.0000e+00 - val_loss: -1.0000 - val_acc: 0.0000e+00
Epoch 62/100
54/54 [==============================] - 0s 6ms/step - loss: -1.0000 - acc: 0.0000e+00 - val_loss: -1.0000 - val_acc: 0.0000e+00
Epoch 63/100
54/54 [==============================] - 0s 6ms/step - loss: -1.0000 - acc: 0.0000e+00 - val_loss: -1.0000 - val_acc: 0.0000e+00
Epoch 64/100
54/54 [==============================] - 0s 6ms/step - loss: -1.0000 - acc: 0.0000e+00 - val_loss: -1.0000 - val_acc: 0.0000e+00
Epoch 65/100
54/54 

In [292]:
model.predict(x_test)


array([[[0.2810425 , 0.28109732, 0.28110966, ..., 0.28116086,
         0.28112018, 0.28110608],
        [0.29935607, 0.29959962, 0.2998051 , ..., 0.29893827,
         0.29907474, 0.29905173],
        [0.31032154, 0.31022978, 0.30992028, ..., 0.31038344,
         0.310261  , 0.3103292 ],
        ...,
        [0.3272982 , 0.32738626, 0.32737046, ..., 0.3272065 ,
         0.32722005, 0.32721117],
        [0.3273597 , 0.32744834, 0.32742724, ..., 0.3272465 ,
         0.32726088, 0.327252  ],
        [0.32740033, 0.32748702, 0.32746175, ..., 0.32726914,
         0.32728425, 0.32727545]],

       [[0.2810425 , 0.28109732, 0.28110966, ..., 0.28116086,
         0.28112018, 0.28110608],
        [0.29935607, 0.29959962, 0.2998051 , ..., 0.29893827,
         0.29907474, 0.29905173],
        [0.31032154, 0.31022978, 0.30992028, ..., 0.31038344,
         0.310261  , 0.3103292 ],
        ...,
        [0.3272982 , 0.32738626, 0.32737046, ..., 0.3272065 ,
         0.32722005, 0.32721117],
        [0.3

In [ ]:
mod = gensim.models.Word2Vec.load('word2vec.bin');
x=input("Enter the message:");
sentend=np.ones((300,),dtype=np.float32) 

sent=nltk.word_tokenize(x.lower())
sentvec = [mod[w] for w in sent if w in mod.wv.vocab]

sentvec[14:]=[]
sentvec.append(sentend)
if len(sentvec)<15:
    for i in range(15-len(sentvec)):
        sentvec.append(sentend) 
sentvec=np.array([sentvec])
    
predictions = model.predict(sentvec)
print (predictions)